<a href="https://colab.research.google.com/github/sinheechan/ML/blob/main/240116_%EC%B4%88%EC%8A%B9%EB%8B%AC_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

문제 1

초승달 데이터셋을 이용한 결정트리 모델을 미세 조정합니다.

교차검증을 사용하는 그리드 탐색 실행

85 ~ 87% 정도의 정확도가 나와야 함

In [ ]:
from sklearn.datasets import make_moons

X_moons, y_moons = make_moons(n_samples=1000, noise=0.4, random_state=2024)
print(len(X_moons), len(y_moons))
print(X_moons[0], print(y_moons[0:10]))

1000 1000
[0 1 0 1 1 0 0 1 1 0]
[-1.08243937  0.56243529] None


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_moons, y_moons, test_size=0.2, random_state=2024)
print(len(X_train), len(y_train))
print(len(y_train), len(y_test))

800 800
800 200


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {'max_leaf_nodes':list(range(2,100)),
          'max_depth':list(range(1, 7)),
          'min_samples_split': [2,3,4]
          }

grid_cv = GridSearchCV(DecisionTreeClassifier(random_state=2024), params, cv=3)

grid_cv.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=2024),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]})

In [ ]:
grid_cv.best_estimator_

DecisionTreeClassifier(max_depth=6, max_leaf_nodes=15, random_state=2024)

In [ ]:
from sklearn.metrics import accuracy_score

pred = grid_cv.predict(X_test)
accuracy_score(y_test, pred)

0.865

문제 2

랜덤 포레스트(Random Forest) 모델을 훈련하는 방법을 살펴봅니다. 랜덤 포레스트는 여러 개의 결정트리로 구성된다.

무작위로 선택된 100개의 초승달 훈련 샘플로 구성된 훈련셋 1,000개를 생성한다. 이를 위해 ShuffleSplit 클래스를 이용한다.
앞서 찾은 최적의 모델을 각 미니 훈련셋에 대해 추가 훈련한 다음 테스트셋에 대한 정확도의 평균값을 계산한다.(결과는 80% 정도)
정확도의 평균이 아닌 1,000개의 모델이 가장 많이 예측하는 값을 예측값으로 사용해보자.(최빈값 계산 - 앙상블 학습)

In [ ]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000    # 결정트리 수. 즉, 결정숲 크기.
n_instances = 100 # 각 결정트리 훈련에 사용되는 훈련셋 크기

mini_sets = []    # 1000개의 결정트리 훈련에 사용될 미니 훈련셋(크기 100)과 타깃셋 튜플들의 리스트 저장

# 1000개의 미니 훈련셋에 포함될 샘플들의 인덱스를 무작위 선택하는 객체 생성
rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances,
                  random_state=2024)

# split 메서드가 X_train 대상으로 1000개의 미니 훈련셋에 포함될 샘플들의 인덱스를 무작위 선택
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index] # 미니 훈련셋
    y_mini_train = y_train[mini_train_index] # 타깃셋

    mini_sets.append((X_mini_train, y_mini_train))

In [ ]:
from sklearn.base import clone # clone() 함수: 모델 복제. 동일 모델을 반복해서 사용하지 않기 위해 사용
import numpy as np

forest = [clone(grid_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)

    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.801145

In [ ]:
# 1000개의 모델이 테스트셋에 대해 예측한 값들로 이뤄진 2차원 어레이
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

# 1000개의 모델이 테스트셋에 대해 예측한 값
for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [ ]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0, keepdims=True) # 최빈값 계산

In [ ]:
print(f"mode: {y_pred_majority_votes}", f"count: {n_votes}", sep='\n')

mode: [[0 1 1 0 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 1 0 1 1 1 1 0 1 0 0 1 0 0 0 1 0 1
  1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 0 0 0 1
  0 1 0 1 1 1 1 0 0 1 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 1 1 0
  0 1 1 1 0 1 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 1 0
  1 0 0 0 0 1 1 1 0 0 0 1 1 0 1 1 1 1 0 1 0 1 1 0 1 0 1 0 1 1 1 1 0 1 0 0
  1 1 0 1 1 1 1 0 0 1 1 1 1 1 0 1 0 0 1 1]]
count: [[891 889 929 946 957 987 928 906 994 534 758 958 921 956 658 909 883 730
  994 916 905 943 949 795 907 882 829 617 788 866 894 866 843 969 704 913
  961 939 926 985 886 816 945 773 990 956 950 899 543 942 503 982 581 800
  982 952 954 991 897 743 960 901 906 943 992 924 876 933 932 909 989 543
  991 570 667 931 943 829 891 955 836 943 983 795 947 796 934 981 933 780
  903 947 948 943 995 983 611 949 858 688 958 697 890 912 994 823 681 956
  625 923 882 835 603 994 839 971 909 955 992 939 944 512 787 899 994 986
  983 967 938 739 948 812 792 946 960 969 971 933 824 9

In [ ]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.855